In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import roc_auc_score
import xgboost as xgb
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK
import mlflow
import mlflow.sklearn

# 데이터 로딩
data = pd.read_csv('../data/dataset.csv')

mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [10]:
# 예측 타겟과 특성 분리
X = data.drop('Default', axis=1)
y = data['Default']

In [11]:
# 범주형과 수치형 컬럼 구분
categorical_cols = ['State', 'BankState', 'NewExist', 'UrbanRural', 'RealEstate']
numerical_cols = ['DisbursementGross', 'GrAppv', 'daysterm']


In [12]:
# 데이터 전처리 파이프라인 구성
preprocessor = ColumnTransformer(transformers=[
    ('num', StandardScaler(), numerical_cols),
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
])

In [13]:
# 데이터 전처리 실행
X_processed = preprocessor.fit_transform(X)

In [14]:
# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.2, random_state=42)

# 하이퍼파라미터 탐색 공간 설정
space = {
    'max_depth': hp.choice('max_depth', range(3, 10)),
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.2),
    'n_estimators': hp.choice('n_estimators', range(50, 200)),
    'gamma': hp.uniform('gamma', 0, 5)
}


In [15]:
# 최적화를 위한 목적 함수 정의
def objective(params):
    # MLflow에 실험 이름 설정
    mlflow.set_experiment("assignment1")

    # 각 하이퍼파라미터 조합별 실험 시작
    with mlflow.start_run(nested=True):
        # XGBoost 모델 초기화 및 훈련
        model = xgb.XGBClassifier(eval_metric='logloss', use_label_encoder=False, **params)
        model.fit(X_train, y_train)

        # 예측 확률 및 ROC-AUC 계산
        probs = model.predict_proba(X_test)[:, 1]
        auc = roc_auc_score(y_test, probs)

        # MLflow에 파라미터 및 메트릭 로깅
        mlflow.log_params(params)
        mlflow.log_metric("roc_auc", auc)

        # 목적 함수 결과 반환 (손실 최소화)
        return {'loss': -auc, 'status': STATUS_OK}


In [16]:
# Hyperopt를 사용한 최적화 실행
trials = Trials()
best_params = fmin(
    fn=objective,
    space=space,
    algo=tpe.suggest,
    max_evals=30,
    trials=trials
)

  0%|          | 0/30 [00:00<?, ?trial/s, best loss=?]

2025/10/10 16:03:40 INFO mlflow.tracking.fluent: Experiment with name 'assignment1' does not exist. Creating a new experiment.



🏃 View run luminous-foal-439 at: http://127.0.0.1:5000/#/experiments/1/runs/d1d5f42bf1184ba884033549e6f9e974

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1

  3%|▎         | 1/30 [00:00<00:04,  5.92trial/s, best loss: -0.9720819720819721]

c:\Users\SSAFY\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\training.py:183: UserWarning: [16:03:40] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)

c:\Users\SSAFY\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\training.py:183: UserWarning: [16:03:40] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



🏃 View run aged-shrike-649 at: http://127.0.0.1:5000/#/experiments/1/runs/38e65b8c44774f21978aa723527d6410

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1                     

🏃 View run sneaky-carp-196 at: http://127.0.0.1:5000/#/experiments/1/runs/50c344e36f8a4c22ae87f8f04cceed5b

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1                    

 10%|█         | 3/30 [00:00<00:03,  6.85trial/s, best loss: -0.979123354123354]

...

🏃 View run bustling-hawk-881 at: http://127.0.0.1:5000/#/experiments/1/runs/0b2261a73ba145a6805f5912f696af00

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1                      

🏃 View run enthused-donkey-328 at: http://127.0.0.1:5000/#/experiments/1/runs/0987b978a3304dffa4cb911bbaf0d35a

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1                      

 90%|█████████ | 27/30 [00:03<00:00,  6.63trial/s, best loss: -0.9792223542223542]

c:\Users\SSAFY\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\training.py:183: UserWarning: [16:03:44] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)

c:\Users\SSAFY\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\training.py:183: UserWarning: [16:03:44] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



🏃 View run mysterious-finch-481 at: http://127.0.0.1:5000/#/experiments/1/runs/84507e57b6564b9abc7cfc6655af65c2

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1                      

🏃 View run shivering-deer-98 at: http://127.0.0.1:5000/#/experiments/1/runs/13a601fc60234b2d90eeef5b4b1fdd52

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1                      

 97%|█████████▋| 29/30 [00:04<00:00,  6.27trial/s, best loss: -0.9792223542223542]

c:\Users\SSAFY\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\training.py:183: UserWarning: [16:03:44] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)

c:\Users\SSAFY\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\training.py:183: UserWarning: [16:03:44] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



🏃 View run serious-ram-906 at: http://127.0.0.1:5000/#/experiments/1/runs/c4850e80f9c84c2e8de64d50be9a7e35

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1                      

100%|██████████| 30/30 [00:04<00:00,  7.18trial/s, best loss: -0.9792223542223542]
